In [1]:
### Correlation

In [2]:
#Dependencies
#conda install reportlab
import pandas as pd
import numpy as np
import matplotlib
import os
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import itertools

In [3]:
#constants
markersize = 2

In [4]:
#report path
report_location = r"C:\Users\EstebanSoto\Documents\Estudio Optico Escarabajos\report"

#code name samples path
code_name_path = r"C:\Users\EstebanSoto\Documents\Estudio Optico Escarabajos\TRANSFLECTANCIA NOV 23\CODE NAME SAMPLES.txt"

#optical properties datapath
reflectance_data_path  = r"C:\Users\EstebanSoto\Documents\Estudio Optico Escarabajos\Artículo Ospina Orozco\1_Reflectancedata.csv"
transmittance_data_path= r"C:\Users\EstebanSoto\Documents\Estudio Optico Escarabajos\Artículo Ospina Orozco\2_Transmittancetopdata.csv"

#transflectance_data_path = r"C:\Users\EstebanSoto\Documents\Estudio Optico Escarabajos\new_data\2023NOV_TRANSFLECTANCE\transflectance\average"

#absorptance_data_path = r"C:\Users\EstebanSoto\Documents\Estudio Optico Escarabajos\clean_data\absorptance"
#Solar spectral irradiance datapath
solar_irradiance_data_path = r"C:\Users\EstebanSoto\Documents\Estudio Optico Escarabajos\reflectivity\ASTM G173-03 (2020).txt"
lunar_irradiance_data_path = r"C:\Users\EstebanSoto\Documents\Estudio Optico Escarabajos\reflectivity\lunar_irradiance_model_MT2009_aprox.txt"

irradiance_data_path = solar_irradiance_data_path

In [5]:
def list_files_in_folder(folder_path):
    files_only_in_folder = []
    
    # Get the list of all files in the folder and its subfolders
    all_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            all_files.append(os.path.join(root, file))
    
    # Get the list of files that are only in the top-level folder
    for file in all_files:
        if os.path.isfile(file) and os.path.dirname(file) == folder_path:
            files_only_in_folder.append(file)
    
    return files_only_in_folder

In [6]:
#transflectance_file_list = list_files_in_folder(transflectance_data_path)
transmittance_file_list = [transmittance_data_path]
#absorptance_file_list = list_files_in_folder(absorptance_data_path)
reflectance_file_list = [reflectance_data_path]
irradiance_file_list = [irradiance_data_path]
for file in transmittance_file_list: 
    print(file)
print(len(transmittance_file_list))


C:\Users\EstebanSoto\Documents\Estudio Optico Escarabajos\Artículo Ospina Orozco\2_Transmittancetopdata.csv
1


In [7]:
def read_dataframes(file_path_list, header_num, measurement_type1, measurement_type2=None): 
    dataframes = []
    
    for file_path in file_path_list:
        if measurement_type2 is None:
            # For a single measurement type
            df = pd.read_csv(file_path, sep='\t', header=header_num, names=['wavelength', measurement_type1]).dropna()
            
            # Replace commas with dots
            df = df.replace(',','.', regex=True)
            
            # Add columns and convert types
            df["file"] = file_path
            df["sample"] = os.path.basename(file_path).replace("-RX", "").replace(".txt", "").replace("_AVERAGE", "").replace("-TX", "").replace("-AX", "")
            df["measurement_type"] = measurement_type1
            df['wavelength'] = df['wavelength'].astype(float)
            df[measurement_type1] = df[measurement_type1].astype(float)
        else:
            # For two measurement types
            df = pd.read_csv(file_path, sep='\t', header=header_num, names=['wavelength', measurement_type1, measurement_type2]).dropna()
            
            # Replace commas with dots
            df = df.replace(',','.', regex=True)
            
            # Add columns and convert types
            df["file"] = file_path
            df["sample"] = os.path.basename(file_path)
            df["measurement_type"] = f"{measurement_type1}_{measurement_type2}"
            df['wavelength'] = df['wavelength'].astype(float)
            df[measurement_type1] = df[measurement_type1].astype(float)
            df[measurement_type2] = df[measurement_type2].astype(float)
        
        # Append the dataframe to the list
        dataframes.append(df)
        dataframe = dataframes[0]
    return dataframe


## Dataframes 

In [8]:
temp_reflectance_df = pd.read_csv(reflectance_data_path, header = 0)

reflectance_df = pd.DataFrame([])
reflectance_df["wavelength"] = temp_reflectance_df["wl"]
reflectance_df["reflectance"] = temp_reflectance_df["aurs1"]

temp_transmittance_df = pd.read_csv(transmittance_data_path, header = 0)

transmittance_df = pd.DataFrame([])
transmittance_df["wavelength"] = temp_transmittance_df["wl"]
transmittance_df["transmittance"] = temp_transmittance_df["aurs1t"]

irradiance_df = read_dataframes(irradiance_file_list, 1 , "tilt_irradiance", "irradiance")

print(reflectance_df )

      wavelength  reflectance
0            400     4.875909
1            401     4.780455
2            402     4.685000
3            403     4.595341
4            404     4.505682
...          ...          ...
1296        1696    53.317500
1297        1697    53.639583
1298        1698    53.961667
1299        1699    54.446548
1300        1700    54.931429

[1301 rows x 2 columns]


## Optical Properties By Range

In [9]:
optical_properties = pd.DataFrame([])

In [10]:
def filter_dataframes_by_range(min_wavelength, max_wavelength, df):

    # Assuming the wavelength column in each DataFrame is named 'wavelength'
    filtered_df = df[(df['wavelength'] >= min_wavelength) & (df['wavelength'] <= max_wavelength)]
    
    
    return filtered_df


In [11]:
def get_optical_property(measurement_type):
    switch_dict = {
        'transmittance': 'transmitivity',
        'reflectance': 'reflectivity',
        'absorptance': 'absorptivity'
    }

    return switch_dict.get(measurement_type, 'optical_property')

In [12]:
# Absorptance

In [13]:
transmittance_df

,wavelength,transmittance
0,300,1.335625
1,302,1.368000
2,304,1.418182
3,306,1.425000
4,308,1.385217
...,...,...
696,1692,40.700000
697,1694,40.381429
698,1696,39.878333
699,1698,39.540000


In [14]:
reflectance_df

,wavelength,reflectance
0,400,4.875909
1,401,4.780455
2,402,4.685000
3,403,4.595341
4,404,4.505682
...,...,...
1296,1696,53.317500
1297,1697,53.639583
1298,1698,53.961667
1299,1699,54.446548


In [15]:
irradiance_df

,wavelength,tilt_irradiance,irradiance,file,sample,measurement_type
0,280.0,4.730000e-23,2.540000e-26,C:\Users\EstebanSoto\Documents\Estudio Optico ...,ASTM G173-03 (2020).txt,tilt_irradiance_irradiance
1,280.5,1.230000e-21,1.090000e-24,C:\Users\EstebanSoto\Documents\Estudio Optico ...,ASTM G173-03 (2020).txt,tilt_irradiance_irradiance
2,281.0,5.690000e-21,6.130000e-24,C:\Users\EstebanSoto\Documents\Estudio Optico ...,ASTM G173-03 (2020).txt,tilt_irradiance_irradiance
3,281.5,1.570000e-19,2.750000e-22,C:\Users\EstebanSoto\Documents\Estudio Optico ...,ASTM G173-03 (2020).txt,tilt_irradiance_irradiance
4,282.0,1.190000e-18,2.830000e-21,C:\Users\EstebanSoto\Documents\Estudio Optico ...,ASTM G173-03 (2020).txt,tilt_irradiance_irradiance
...,...,...,...,...,...,...
1723,2610.0,0.000000e+00,0.000000e+00,C:\Users\EstebanSoto\Documents\Estudio Optico ...,ASTM G173-03 (2020).txt,tilt_irradiance_irradiance
1724,2615.0,0.000000e+00,0.000000e+00,C:\Users\EstebanSoto\Documents\Estudio Optico ...,ASTM G173-03 (2020).txt,tilt_irradiance_irradiance
1725,2620.0,0.000000e+00,0.000000e+00,C:\Users\EstebanSoto\Documents\Estudio Optico ...,ASTM G173-03 (2020).txt,tilt_irradiance_irradiance
1726,2625.0,0.000000e+00,0.000000e+00,C:\Users\EstebanSoto\Documents\Estudio Optico ...,ASTM G173-03 (2020).txt,tilt_irradiance_irradiance


In [16]:
absorptance_df = pd.DataFrame([])
absorptance_df["wavelength"] = transmittance_df["wavelength"]
absorptance_df["absorptance"] = 100-transmittance_df["transmittance"]-reflectance_df["reflectance"]
absorptance_df

,wavelength,absorptance
0,300,93.788466
1,302,93.851545
2,304,93.896818
3,306,93.979659
4,308,94.109101
...,...,...
696,1692,-1.279167
697,1694,-0.944762
698,1696,-0.425833
699,1698,-0.063750


In [17]:
def optical_properties_by_range(min_wv, max_wv, optical_property_df, irradiance_df, measurement_type):
    properties=[]
    optical_property_df = filter_dataframes_by_range(min_wv, max_wv, optical_property_df)
    
    irradiance_df = filter_dataframes_by_range(min_wv, max_wv, irradiance_df)
    
    # Calculate total irradiance
    sum_irradiance = irradiance_df['irradiance'].sum()
    print(sum_irradiance)
    # Interpolate irradiance data to match the wavelength values of optical property data
    #interpolated_irradiance = np.interp(optical_property_df['wavelength'], irradiance_df['wavelength'], irradiance_df['normal_irradiance'], left=0, right=0)

    # Calculate the multiplied values
    result_intensity = optical_property_df[measurement_type] * irradiance_df['irradiance']

    # Create a new DataFrame with the multiplied values
    result_dataset = pd.DataFrame({'wavelength': optical_property_df['wavelength'], measurement_type: result_intensity})

    # Calculate sum of optical property
    sum_optical_property = result_dataset[measurement_type].sum()
    #print(sum_optical_property[0])

    # Calculate the optical property value
    optical_property = get_optical_property(measurement_type)
    optical_property_value = sum_optical_property / sum_irradiance
    result =  {optical_property: optical_property_value}
    properties.append(result)

    properties_df = pd.DataFrame(properties, columns=[ optical_property]).sort_values(by=optical_property, ascending=False)
    return properties_df

# Reflectivity

## General Reflectivity

In [18]:
min_wv = 400
max_wv = 1700

type(irradiance_df)

reflectivity= optical_properties_by_range(min_wv, max_wv, reflectance_df, irradiance_df, "reflectance")


816.125


In [19]:
min_wv = 400
max_wv = 1700

transmitivity= optical_properties_by_range(min_wv, max_wv, transmittance_df, irradiance_df, "transmittance")

816.125


In [20]:
min_wv = 400
max_wv = 1700

absorptivity = 100 - reflectivity["reflectivity"] - transmitivity["transmitivity"]

In [21]:
print(reflectivity) 
print(transmitivity)
print(absorptivity) 

   reflectivity
0     58.764612
   transmitivity
0      26.246898
0    14.98849
dtype: float64
